<a href="https://colab.research.google.com/github/srivatsan88/YouTubeLI/blob/master/TFX_Visualize_Distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Link to supporting video for below notebook walkhthrough - https://www.youtube.com/watch?v=eGIG_qHgQ08

In [0]:
from __future__ import print_function
import sys,tempfile, urllib, os

Install tensorflow data validation library

In [0]:
!pip install -q tensorflow_data_validation
import tensorflow_data_validation as tfdv

print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 0.14.1


In [0]:
!pip install pandas-profiling

Create file in local system to store the downloaded dataset

In [0]:
BASE_DIR = '/tmp'
OUTPUT_FILE = os.path.join(BASE_DIR, 'churn_data.csv')

Download the Watson Telecom dataset and store it in local disk

In [0]:
churn_data=urllib.request.urlretrieve('https://raw.githubusercontent.com/srivatsan88/YouTubeLI/master/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv', OUTPUT_FILE)


Do not worry about the dataset details for now. Let us quickly run Tensorflow Datavalidation (TFDV) component to generate statistics on the file

In [0]:
train_stats = tfdv.generate_statistics_from_csv(data_location=OUTPUT_FILE)

Visualize the generated stats. If you see the visualization. TensorFlow Data Validation provides tools for visualizing the distribution of feature values. By examining these distributionsyou can catch common problems with data. One quick observation is SeniorCitizen column below has around 84% zeros. Play around with different chart below and also in case if you want to search into any particular feature

In [0]:
tfdv.visualize_statistics(train_stats)

Let us now create schema for our data using infer_schema method. Schema defines constraints for the data that are relevant for ML. Example constraints include the data type of each feature, whether it's numerical or categorical, or the frequency of its presence in the data. For categorical features the schema also defines the domain - the list of acceptable values. Since writing a schema can be a tedious task, especially for datasets with lots of features, TFDV provides a method to generate an initial version of the schema based on the descriptive statistics.

Getting the schema right is important because the rest of our production pipeline will be relying on the schema that TFDV generates to be correct. The schema also provides documentation for the data

In [0]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'TechSupport',STRING,required,,'TechSupport'
'MonthlyCharges',FLOAT,required,,-
'Churn',STRING,required,,'Churn'
'Contract',STRING,required,,'Contract'
'tenure',INT,required,,-
'SeniorCitizen',INT,required,,-
'StreamingTV',STRING,required,,'StreamingTV'
'PaymentMethod',STRING,required,,'PaymentMethod'
'OnlineBackup',STRING,required,,'OnlineBackup'


,Values
Domain,
'TechSupport',"'No', 'No internet service', 'Yes'"
'Churn',"'No', 'Yes'"
'Contract',"'Month-to-month', 'One year', 'Two year'"
'StreamingTV',"'No', 'No internet service', 'Yes'"
'PaymentMethod',"'Bank transfer (automatic)', 'Credit card (automatic)', 'Electronic check', 'Mailed check'"
'OnlineBackup',"'No', 'No internet service', 'Yes'"
'PaperlessBilling',"'No', 'Yes'"
'OnlineSecurity',"'No', 'No internet service', 'Yes'"
'InternetService',"'DSL', 'Fiber optic', 'No'"


Let us now load the file downloaded earlier in pandas dataframe and split the dataset to compare distirbution and schema against each other

In [0]:
import pandas as pd
import pandas_profiling

In [0]:
churn_df = pd.read_csv(OUTPUT_FILE)

In [0]:
churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [0]:
profile = pandas_profiling.ProfileReport(churn_df.head(100))


In [0]:
profile

Number of variables,21
Number of observations,100
Total Missing (%),0.0%
Total size in memory,16.5 KiB
Average record size in memory,168.8 B
Numeric,2
Categorical,16
Boolean,1
Date,0
Text (Unique),2
Rejected,0
